In [ ]:
#@title Setup
%tensorflow_version 1.x
un = 'githubuser' #@param {type:"string"}
pw = 'githubpassword' #@param {type:"string"}
branch = "tfmesh"
url = "https://" + un + ":" + pw + "@github.com/ConnorJL/GPTNeo"
!git clone -b $branch $url
!pip3 install tensorflow==1.15.2 regex google-api-python-client oauth2client requests tqdm ftfy newspaper3k
%cd GPTNeo


In [ ]:
from google.colab import auth
auth.authenticate_user()

You'll need to edit all the bucket paths (gs://...) below to point to your own cloud bucket.

The below just downloads a small sample dataset - this will not result in a good model.


In [ ]:
# get dataset
download = False
if download:
  !wget -P datasets/ https://storage.googleapis.com/connors-datasets/bundestag/bundestag_0.tfrecords
  !gsutil cp /content/GPTNeo/datasets/bundestag_0.tfrecords $data_path
!gsutil ls gs://datasets_storage_1/datasets/bundestag

Before running the final cell - you'll need to edit the config below to provide paths to your own bucket / data

In [ ]:
%%writefile configs/colab.json

{
    "n_head": 8, 
    "encoder_path": "gs://datasets_storage_1/models/encoder",
    "n_vocab": 50257,
    "embed_dropout": 0.1,
    "lr": 0.00025,
    "warmup_steps": 0,
    "beta1": 0.9,
    "beta2": 0.98,
    "epsilon": 1e-9,
    "opt_name": "adam",
    "weight_decay": 0.01,
    "train_batch_size": 32,
    "attn_dropout": 0.1,
    "train_steps": 10000,
    "eval_steps": 0,
    "max_steps": 500000,
    "data_path": "gs://datasets_storage_1/datasets/bundestag",
    "res_dropout": 0.1,
    "predict_batch_size": 1,
    "eval_batch_size": 32,
    "iterations": 500,
    "n_embd": 768,
    "datasets": [["bundestag_*.tfrecords", "", 10, "random_sample", 1.0]],
    "model": "GPT2",
    "model_path": "gs://datasets_storage_1/models/GPTNeo_prettybig",
    "n_ctx": 128,
    "predict_path": "logs/predictions.txt",
    "n_layer": 12,
    "scale_by_depth": true,
    "scale_by_in": true,
    "fixed_attn_block_size": 128,
    "layer_offset": 16,
    "local": true,
    "mesh_shape": "all:8",
    "layout": "heads:all",
    "precision": "float32"
}


In [ ]:
# this deletes all the old ckpts - you'll need to enter your own bucket path
!gsutil rm -r gs://datasets_storage_1/models/GPTNeo_prettybig/ 
# a real model will need steps_per_ckpt to be much larger - this speeds up testing
!python3 gpt2_mesh.py --model_params configs/GPT_NEO_TEST.json --steps_per_checkpoint 20 #